In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [3]:
def experimental_isotherms(p_range, v_range, T_range, fixed_p, fixed_T):

    if fixed_T:
    
        theo_data = get_relative_isotherms(v_range, T_range)
        expe_data = []

        v_limits = []
        p_limits = []

        p_range = np.linspace(0.001, 1.0, num=10000)
        pressures, v_isopressure_limits = find_real_fixed_T(p_range, T_range)

        for i in range(len(theo_data)):

            p_expe = []

            if i < len(v_isopressure_limits):

                    v_lim = v_isopressure_limits[i]

                    if len(v_lim) > 1:
                        for j in range(len(v_range)):

                            if v_range[j] > v_lim[0] and v_range[j] < v_lim[1]:
                                p_expe.append(pressures[i])

                            else:
                                p_expe.append(theo_data[i][j])

                        v_limits = np.append(v_limits, [v_lim[0], v_lim[1]])
                        p_limits = np.append(p_limits, [pressures[i], pressures[i]])

                    else:
                        p_expe = theo_data[i]
                        v_limits = np.append(v_limits, [1.0])
                        p_limits = np.append(p_limits, [1.0])

            else:

                p_expe = theo_data[i]

            expe_data.append(p_expe)      

        return expe_data, theo_data, p_limits, v_limits, T_range 
    
    elif fixed_p:
        
        tenperatures, v_isopressure_limits = find_real_fixed_p(p_range, T_range)

        theo_data = get_relative_isotherms(v_range, tenperatures)
        
        expe_data = []

        v_limits = []
        p_limits = []
        
        for i in range(len(theo_data)):

            p_expe = []

            if i < len(v_isopressure_limits):

                    v_lim = v_isopressure_limits[i]

                    if len(v_lim) > 1:
                        for j in range(len(v_range)):

                            if v_range[j] > v_lim[0] and v_range[j] < v_lim[1]:
                                p_expe.append(p_range[i])

                            else:
                                p_expe.append(theo_data[i][j])

                        v_limits = np.append(v_limits, [v_lim[0], v_lim[1]])
                        p_limits = np.append(p_limits, [p_range[i], p_range[i]])

                    else:
                        p_expe = theo_data[i]
                        v_limits = np.append(v_limits, [1.0])
                        p_limits = np.append(p_limits, [1.0])

            else:

                p_expe = theo_data[i]

            expe_data.append(p_expe)      

        return expe_data, theo_data, p_range, v_limits, tenperatures

In [4]:
def get_relative_isotherms(v_range, T_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """

    isotherms = []

    for T in T_range:
        p_R = []
        for v in v_range:
            val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
            p_R = np.append(p_R, val)

        isotherms.append(p_R)

    return isotherms

In [5]:
def find_real_fixed_T(p_range, T):
    
    eps = 1e-3 #1.0/p_range.size
    
    for p in p_range:
        
        roots = get_roots(p, T)
        if len(roots) == 3:
            v_range = [roots[0], roots[2]]
            area = defined_integral(p, v_range, T)
            if abs(area) < eps:
                return p, v_range

    return None

In [6]:
def get_roots(p, T):
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        if np.isreal(root):
            root = np.real(root)
            if root > 0:
                roots_in_range.append(root)
    #print(roots_in_range) 
    roots_in_range.sort()
    
    return roots_in_range

In [7]:
def defined_integral(p_0, v_range, T):
    v_0, v_1 = v_range[0], v_range[1]
    return p_undefined_integral(p_0, v_1, T) - p_undefined_integral(p_0, v_0, T)

In [8]:
def p_undefined_integral(p_0, v_0, T):
    
    return 8.0/3.0 * T *np.log(v_0 - 1.0/3.0) + 3.0/v_0 - p_0*v_0

In [9]:
def Gibbs(G_1, G_2, G_3, G_4, phi):
    
    gibbs = []
    
    for f in phi:
        gibbs = np.append(gibbs, G_1*f + G_2*f**2 + G_3*f**3 + G_4*f**4)
        
    return gibbs

In [22]:
tt_box = widgets.HBox([])

def hover_handler(self, content):

    g = {0:[-40, -40, 1.0, 20],
         1:[0, -40, 0, 40],
         2:[40, -40, -1.0, 20],
         3:[0, 40, 0, 20]}

    params = g.get(content.get('data').get('index'))

    gibbs = Gibbs(G_1=params[0], G_2=params[1], G_3=params[2], G_4=params[3], phi = phi)

    scale_y_11B_002.min = min(gibbs) - 100
    scale_y_11B_002.max = 200
    gibbs_lines.y = Gibbs(G_1=params[0], G_2=params[1], G_3=params[2], G_4=params[3], phi = phi)
    tt_box.children = [fig_11B_002,]

In [23]:
v_values = np.linspace(start=0.4, stop=5, num=1000)
T_values = np.linspace(start=0.85, stop=1.0, num=15)

p_values = get_relative_isotherms(v_values, T_values)


trans_pressures = []

for i in range(len(T_values)):
    T = T_values[i]
    p = p_values[i]
    
    if T < 1.0:
        p_trans, v_range = find_real_fixed_T(p, T)

        if p_trans is not None:
            trans_pressures = np.append(trans_pressures, p_trans)

                
    elif T == 1.0:
        
            trans_pressures = np.append(trans_pressures, [1.0])

j = 12
scatter_tenp = [T_values[j]-0.05, T_values[j], T_values[j]+0.05, 1.0]
scatter_pres = [trans_pressures[j], trans_pressures[j], trans_pressures[j], 1.0]

In [25]:
phi = np.linspace(-12, 5, 100)
gibbs = Gibbs(G_1=1.0, G_2=-20.0, G_3=200, G_4=20.0, phi = phi)

scale_x = bqs.LinearScale(min = min(trans_pressures), max = 1.2)
scale_y = bqs.LinearScale(min = min(T_values), max = 1.2)

fig_111_000 = bq.Figure(title='T vs p (fixed v, reduced variables)',
                marks=[],
                axes=[],
                animation_duration=0,
                layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=60, right=30),
                toolbar = True
    )

axis_x = bqa.Axis(scale=scale_x,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=np.linspace(scale_x.min, scale_x.max, 5),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='p',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(scale=scale_y,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=np.linspace(0.0, 2.0, 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_111_000.axes = [axis_x, axis_y]

pressures_lines = bqm.Lines(
    name = 'labels',
    x = trans_pressures,
    y = T_values,
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=['labels']
) 

critic_point = bqm.Scatter(
    name = '',
    x = [1.0],
    y = [1.0],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=[]
) 

scatter_points = bqm.Scatter(
    name = '',
    x = scatter_pres,
    y = scatter_tenp,
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=[],
    tooltip = tt_box
)

scatter_points.on_hover(hover_handler)

fig_111_000.marks = [pressures_lines,
                     scatter_points]

scale_x_11B_002 = bqs.LinearScale(min = -4, max = 4)
scale_y_11B_002 = bqs.LinearScale(min = min(gibbs), max = 10000)

fig_11B_002 = bq.Figure(title='G (T, phi)',
                marks=[],
                axes=[],
                animation_duration=0,
                layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=60, right=30),
                toolbar = True
    )

axis_x_11B_002 = bqa.Axis(scale=scale_x_11B_002,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=0,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='p',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_11B_002 = bqa.Axis(scale=scale_y_11B_002,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=0,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_11B_002.axes = [axis_x_11B_002, axis_y_11B_002]

gibbs_lines = bqm.Lines(
    name = '',
    x = phi,
    y = gibbs,
    scales = {'x': scale_x_11B_002, 'y': scale_y_11B_002}, 
    #opacities = [1.0],
    visible = True,
    colors = ['blue'],
    names = [],
    labels=['']
) 

fig_11B_002.marks = [gibbs_lines]
fig_111_000

Figure(axes=[Axis(grid_color='#8e8e8e', grid_lines='none', label='p', label_offset='50px', scale=LinearScale(m…